# FIRE 

This notebook will try to answer the following questions:

- Given my current savings and no income, how many months will my savings last


## Assumptions

- The annual return on the stock investment (assumption ETF) is an average and is applied monthly. 
The assumption is a long term.
- The bond investment has a fixed return rate that's applied monthly. In reality that should be close to inflation rate.
- The monthly income is constant and is added to cash every month. It's also a simplification as in reality the income is not constant
- The monthly expenses are constant but the inflation rate is applied. In reality, the expenses won't be constant as they change depending on many factors.
- The inflation rate is constant
- When the cash is depleted, the investment is used to cover the expenses. The investment is used in the following order: cash, bond, stock, real estate. 
- Simulation ends when the wealth is depleted and no more assets are available to cover the expenses.


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [2]:
import datetime
from fire.simulations import FireSimulation, run_simulation, InvestmentProperty
import pandas as pd
import plotly.express as px
from decimal import Decimal


In [3]:
from decimal import Decimal


init = FireSimulation(
    stock_investments=Decimal("100000"),
    bonds_investments=Decimal("100000"),
    cash=Decimal("100000"),
    investment_properties=[
        InvestmentProperty(
            mortgage_left=Decimal("292000"),
            mortgage_rate=Decimal("7.66"),
            mortgage_months=202,
            market_value=Decimal("520000"),
            monthly_income=Decimal("2100"),
        )
    ],
    stock_return_rate=Decimal("0.05"),
    bonds_return_rate=Decimal("0.04"),
    monthly_expenses=Decimal("16000"),
    annual_inflation_rate=Decimal("0.04"),
    monthly_income=Decimal("0"),
    annual_property_appreciation_rate=Decimal("0.02"),
    invest_cash_surplus=True,
    invest_cash_threshold=Decimal("50000"),
    invest_cash_surplus_strategy='80-20',
    date=datetime.datetime.fromisoformat('2024-04-01')
)

# simulate for next X years
years = 15
expected_number_of_months = years * 12
simulation = run_simulation(init, years*12)

In [4]:

df = pd.DataFrame([s.to_dict() for s in simulation])

number_of_records = len(df)
last_record = df.iloc[-1]
df


,stock_investments,bonds_investments,cash,monthly_expenses,monthly_income,date,stock_return_rate,investment_properties,bonds_return_rate,annual_inflation_rate,...,annual_property_appreciation_rate,invest_cash_surplus,invest_cash_threshold,invest_cash_surplus_strategy,liquid_wealth,wealth_inc_properties,properties_monthly_mortgage,properties_market_value,properties_monthly_income,properties_net_cash_value
0,100000.00,100000.00,100000.00,16000.00,0.0,2024-04-01,0.05,"[{'market_value': 520000, 'monthly_income': 21...",0.04,0.04,...,0.02,True,50000.0,80-20,300000.00,528000.00,2576.48,520000.00,2100.0,228000.00
1,129254.00,107542.67,50000.00,16053.33,0.0,2024-05-01,0.05,"[{'market_value': 520866.67, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,286796.67,516375.89,2576.48,520866.67,2100.0,229579.22
2,129792.56,107901.15,35993.16,16106.84,0.0,2024-06-01,0.05,"[{'market_value': 521734.78, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,273686.87,504851.30,2576.48,521734.78,2100.0,231164.43
3,130333.36,108260.82,21932.63,16160.53,0.0,2024-07-01,0.05,"[{'market_value': 522604.34, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,260526.81,493282.47,2576.48,522604.34,2100.0,232755.66
4,130876.42,108621.69,7818.23,16214.40,0.0,2024-08-01,0.05,"[{'market_value': 523475.35, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,247316.34,481669.29,2576.48,523475.35,2100.0,234352.95
5,131421.74,102633.54,0.00,16268.45,0.0,2024-09-01,0.05,"[{'market_value': 524347.81, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,234055.28,470011.61,2576.48,524347.81,2100.0,235956.33
6,131969.33,88752.97,0.00,16322.68,0.0,2024-10-01,0.05,"[{'market_value': 525221.72, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,220722.30,458288.12,2576.48,525221.72,2100.0,237565.82
7,132519.20,74771.72,0.00,16377.09,0.0,2024-11-01,0.05,"[{'market_value': 526097.09, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,207290.92,446472.39,2576.48,526097.09,2100.0,239181.47
8,133071.36,60689.28,0.00,16431.68,0.0,2024-12-01,0.05,"[{'market_value': 526973.92, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,193760.64,434563.94,2576.48,526973.92,2100.0,240803.30
9,133625.82,46505.13,0.00,16486.45,0.0,2025-01-01,0.05,"[{'market_value': 527852.21, 'monthly_income':...",0.04,0.04,...,0.02,True,50000.0,80-20,180130.95,422562.30,2576.48,527852.21,2100.0,242431.35


In [5]:
print(f"Without any income and burning only your investments, you can survive for {number_of_records} months")



Without any income and burning only your investments, you can survive for 38 months


In [7]:
# I want to use plotly to create a nice stacked bar chart of bonds and stock investments
# with a line chart showing monthly expenses

fig = px.bar(df, x=df['date'], y=[
        "properties_net_cash_value",
        "stock_investments",
        "bonds_investments",
        "cash",
    ], title='Wealth over time')
fig.add_scatter(x=df['date'], y=df['monthly_expenses'], mode='lines', name='Monthly expenses')
fig.add_scatter(x=df['date'], y=df['wealth_inc_properties'], mode='lines', name='Wealth with net properties')

fig.show()
